In [1]:
import pandas as pd
from datetime import datetime

from lifting_cast import LiftingCast
from aws import DynamoLifter, DynamoLifterUpdate


Initializing environment


In [2]:
L = LiftingCast()
meets = L.fetch_meets()
lifters = L.fetch_lifters(meets[:10])




begin fetch_lifters
begin fetch_lifters iteration - 10 meets
fetched 0 lifters for 2024 CPU Central Championships - Day 2
fetched 0 lifters for TSS-UTSA Invitational #12
fetched 89 lifters for 2024 CPU Central Championships - Day 1
fetched 37 lifters for 8th Annual Powerlifting America Southwest Florida Open Championship (2024-FL-02)
fetched 43 lifters for Winneconne Last Chance Meet
fetched 0 lifters for 2024 Special Olympics Texas Winter Games
fetched 34 lifters for Independence Gilmanton WHSPA Regional
fetched 14 lifters for The Real McCloy Bench Press Only
fetched 3 lifters for Pro League Uttarakhand championship 2024
fetched 234 lifters for THSPA AGGIE SHOWDOWN 2024 V2
fetch_lifters iteration finished. Failed: 0
end fetch_lifters


In [3]:

scraped_lifters = pd.DataFrame(lifters)

# lifters that we are currently storing
print("Fetching lifters from dynamo")
aws_lifter = DynamoLifter()
stored_lifters = aws_lifter.get_lifters()


Fetching lifters from dynamo


c:\Users\Gunther\Desktop\lifting-lookup\lifting-lookup-refresh\.venv\Lib\site-packages\awswrangler\dynamodb\_read.py:410: UserWarning: Attempting DynamoDB Scan operation with arguments:
{'ConsistentRead': False}
  warnings.warn(
c:\Users\Gunther\Desktop\lifting-lookup\lifting-lookup-refresh\.venv\Lib\site-packages\awswrangler\_distributed.py:104: FutureWarning: promote has been superseded by mode='default'.
  return cls.dispatch_func(func)(*args, **kw)


In [4]:

# create two dataframes for each action we need to do
lifters_to_delete = pd.DataFrame()
lifters_to_insert = pd.DataFrame()
if "lifter_id" in scraped_lifters.columns and "lifter_id" in stored_lifters.columns:
    lifters_to_delete = stored_lifters[
        ~stored_lifters.lifter_id.isin(scraped_lifters.lifter_id)
    ]
    lifters_to_insert = scraped_lifters[
        ~scraped_lifters.lifter_id.isin(stored_lifters.lifter_id)
    ]



In [10]:
from dotenv import load_dotenv
import os
import awswrangler as wr

print("Initializing environment")
load_dotenv()

class VIPLifterSubscription:
    def __init__(self):
        self.vip_lifter_subscription_table_name = os.environ.get("AWS_DYNAMO_VIP_LIFTER_SUBSCRIPTION_TABLE_NAME")
        
    def get_vip_lifter_subscriptions(self):
        return wr.dynamodb.read_items(
            table_name=self.vip_lifter_subscription_table_name,
            allow_full_scan=True,
            as_dataframe=False
        )

VIP = VIPLifterSubscription()
subs = VIP.get_vip_lifter_subscriptions()



Initializing environment


c:\Users\Gunther\Desktop\lifting-lookup\lifting-lookup-refresh\.venv\Lib\site-packages\awswrangler\dynamodb\_read.py:410: UserWarning: Attempting DynamoDB Scan operation with arguments:
{'ConsistentRead': False}
  warnings.warn(


In [24]:
datetime.now().date()

datetime.date(2024, 2, 15)

In [14]:

subscription_df = pd.DataFrame(subs)
subscription_df = subscription_df.explode('subscription_list')
subscription_df.rename(columns={"subscription_list": "lifter_name"}, inplace=True)
subscription_df = subscription_df[subscription_df["lifter_name"].notna()]


In [38]:
import re
from datetime import datetime

def scrub_lifter_lifter_name(lifter):
    name = lifter["lifter_name"]
    scrubbed_name = re.sub(r'[\d\s-]+', '', name).lower()
    return scrubbed_name

lifters_to_insert["scrubbed_lifter_name"] = lifters_to_insert.apply(scrub_lifter_lifter_name, axis=1)
subscription_df["scrubbed_lifter_name"] = subscription_df.apply(scrub_lifter_lifter_name, axis=1)

notifications = subscription_df.merge(lifters_to_insert, on='scrubbed_lifter_name')
notifications['meet_date'] = pd.to_datetime(notifications['meet_date'], format='%m/%d/%Y')
notifications = notifications[notifications['meet_date'] >= datetime.now()]


C:\Users\Gunther\AppData\Local\Temp\ipykernel_1968\1921409407.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lifters_to_insert["scrubbed_lifter_name"] = lifters_to_insert.apply(scrub_lifter_lifter_name, axis=1)


In [39]:
# emails_df = notifications.groupby('subscriber_email')['lifter_name_x'].apply(list)
emails_dict = {group_key: group_rows.to_dict(orient='records') for group_key, group_rows in notifications.groupby('subscriber_email')}


In [43]:
import boto3
email_client = boto3.client('ses')

def generate_email_html(subscriber_email, lifter_notifications):
    body = f"""
    <p>Greetings {subscriber_email},</p>
    <p>This notification is to alert you of the following lifters registerting for a meet:</p>
    """
    for lifter in lifter_notifications:
        body += f"""<li>{lifter['lifter_name_x']} - <a href="https://liftingcast.com/meets/{lifter['meet_id']}/lifter/{lifter['lifter_id']}/info">{lifter['meet_name']}</a></li>"""
    
    body += f"""
    <p>To update your VIP Lifter Notification settings, please visit <a href="liftinglookup.com/account/vip">LiftingLookup</a>.
    You can also reply directly to this email with any issues or concerns.</p>
    """
    return body


for subscriber_email, lifter_notifications in emails_dict.items():
    send_args = {
        "Source": "vip-notification@liftinglookup.com",
        "Destination": {
            "ToAddresses":  [subscriber_email],
            "BccAddresses": ["vip-notification@liftinglookup.com"]
        },
        "Message": {
            "Subject": {"Data": "LiftingLookup VIP Lifter Notification"},
            "Body": {"Html": {"Data": generate_email_html(subscriber_email, lifter_notifications)}}
        }
    }
    email_client.send_email(**send_args)
